In [1]:
import boto3
import sagemaker
import pandas as pd

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
account_id = boto3.client("sts").get_caller_identity().get("Account")

sm = boto3.Session().client(service_name="sagemaker", region_name=region)

In [2]:
#set private path
private_path = "s3://{}/ads-508-anime/".format(bucket)
print(private_path)





s3://sagemaker-us-east-1-229141875778/ads-508-anime/


In [3]:


ingest_create_athena_db_passed = False



In [4]:
%store -r private_path

In [5]:
try:
    private_path
except NameError:
    print("*****************************************************************************")
    print("[ERROR] PLEASE RE-RUN THE PREVIOUS COPY TSV TO S3 NOTEBOOK ******************")
    print("[ERROR] THIS NOTEBOOK WILL NOT RUN PROPERLY. ********************************")
    print("*****************************************************************************")

In [6]:
print (private_path)

s3://sagemaker-us-east-1-229141875778/ads-508-anime


In [7]:
from pyathena import connect

In [8]:
database_name = "myanimelist"

In [9]:
# Set S3 staging directory -- this is a temporary directory used for Athena queries
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)

In [10]:
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [11]:


statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)



CREATE DATABASE IF NOT EXISTS myanimelist


In [12]:
pd.read_sql(statement, conn)

""


In [13]:
statement = "SHOW DATABASES"

df_show = pd.read_sql(statement, conn)
df_show.head(5)

,database_name
0,anime
1,default
2,dsoaws
3,myanimelist


In [14]:


if database_name in df_show.values:
    ingest_create_athena_db_passed = True



In [15]:
%store ingest_create_athena_db_passed

Stored 'ingest_create_athena_db_passed' (bool)


In [16]:
%store

Stored variables and their in-db values:
auto_ml_job_name                                      -> 'automl-dm-09-16-23-28'
autopilot_endpoint_arn                                -> 'arn:aws:sagemaker:us-east-1:229141875778:endpoint
autopilot_endpoint_name                               -> 'automl-dm-ep-09-17-20-25'
autopilot_model_arn                                   -> 'arn:aws:sagemaker:us-east-1:229141875778:model/au
autopilot_model_name                                  -> 'automl-dm-model-09-17-20-24'
autopilot_train_s3_uri                                -> 's3://sagemaker-us-east-1-229141875778/data/amazon
balanced_bias_data_jsonlines_s3_uri                   -> 's3://sagemaker-us-east-1-229141875778/bias-detect
balanced_bias_data_s3_uri                             -> 's3://sagemaker-us-east-1-229141875778/bias-detect
bias_data_s3_uri                                      -> 's3://sagemaker-us-east-1-229141875778/bias-detect
comprehend_train_s3_uri                               -> 's

In [17]:
#create athena table in Glue


import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name



In [18]:


ingest_create_athena_table_tsv_passed = False



In [19]:
%store -r ingest_create_athena_db_passed

In [20]:


try:
    ingest_create_athena_db_passed
except NameError:
    print("++++++++++++++++++++++++++++++++++++++++++++++")
    print("[ERROR] YOU HAVE TO RUN ALL PREVIOUS NOTEBOOKS.  You did not create the Athena Database.")
    print("++++++++++++++++++++++++++++++++++++++++++++++")



In [21]:
print(ingest_create_athena_db_passed)

True


In [29]:
# SQL statement to execute
animelist_table = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         user_id integer,
         anime_id integer,
         rating integer,
         watching_status integer,
         watched_episodes integer
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\\t' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(
    database_name, 'animelist', private_path
)

print(statement)

CREATE EXTERNAL TABLE IF NOT EXISTS myanimelist.animelist.csv(
         user_id integer,
         anime_id integer,
         rating integer,
         watching_status integer,
         watched_episodes integer
) ROW FORMAT DELIMITED FIELDS TERMINATED BY '\t' LINES TERMINATED BY '\n' LOCATION 's3://sagemaker-us-east-1-229141875778/ads-508-anime'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')


In [30]:


import pandas as pd

pd.read_sql(animelist_table, conn)



""


In [41]:
anime_table = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         MAL_ID int, Name string, Score int, Genres string, English_name string, Japanese_name string, Type string, 
         Episodes int, Aired string, Premiered string, Producers string, 
         Licensors string, Studios string, Source string, Duration string, Rating string, Ranked int,
         Popularity int, Members int, Favorites int, Watching int, Completed int, 
         On_Hold int, Dropped int, Plan_to_Watch int, 
         Score_10 int, Score_9 int, Score_8 int, Score_7 int, Score_6 int, Score_5 int, Score_4 int, Score_3 int, Score_2 int, Score_1 int
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ',' LINES TERMINATED BY '\\n' LOCATION '{}'
TBLPROPERTIES ('compressionType'='gzip', 'skip.header.line.count'='1')""".format(database_name, 'anime.csv', private_path)
pd.read_sql(anime_table, con=conn)

""


In [42]:
pd.read_sql('SELECT * FROM %s.%s LIMIT 20' % (database_name,'anime_table'), con=conn)

DatabaseError: Execution failed on sql: SELECT * FROM myanimelist.anime_table LIMIT 20
SYNTAX_ERROR: line 1:15: Table awsdatacatalog.myanimelist.anime_table does not exist
unable to rollback